# Импорт библиотек

In [34]:
import re

import pandas as pd
import nltk
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import numpy as np

from tqdm import tqdm

nltk.download("punkt")

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
data = pd.read_csv("../data/all_data.csv")

In [36]:
data.describe()

,text,title
count,29815,29815
unique,29815,26860
top,протопоп аввакум написал житие по благословени...,классика зарубежного рассказа
freq,1,26


# Разделение данных

In [37]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

In [38]:
len(train_df), len(val_df)

(23852, 5963)

In [39]:
val_df.describe()

,text,title
count,5963,5963
unique,5963,5786
top,прокладка под голову . ссср сальников сергей с...,сергей степанов. постлюбовь. новый роман
freq,1,6


Заметим, что в валидацию попали несколько текстов с одинаковыми названиями. Это не очень хорошо, так как может исказить результаты оценки модели.

Переделаем разделение.

In [40]:
def split_with_controlled_test_size(data, target_test_size=0.2, random_state=42):
    np.random.seed(random_state)

    title_groups = data.groupby('title').apply(lambda x: x.index.tolist()).to_dict()

    unique_titles = list(title_groups.keys())
    np.random.shuffle(unique_titles)

    train_indices = []
    test_indices = []

    target_test_count = int(len(data) * target_test_size)

    for title in unique_titles:
        indices = title_groups[title]

        if len(test_indices) < target_test_count:
            test_idx = np.random.choice(indices, 1)[0]
            test_indices.append(test_idx)
            train_indices.extend([idx for idx in indices if idx != test_idx])
        else:
            train_indices.extend(indices)

    return data.iloc[train_indices], data.iloc[test_indices]

train_df, val_df = split_with_controlled_test_size(data)

/var/folders/wl/_8h74t5s5hz2ntrny0kjx_pw0000gn/T/ipykernel_39747/2556919616.py:4: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [41]:
print(f"Общий размер данных: {len(data)}")
print(f"Тренировочная выборка: {len(train_df)} записей ({len(train_df)/len(data)*100:.1f}%)")
print(f"Валидационная выборка: {len(val_df)} записей ({len(val_df)/len(data)*100:.1f}%)")

Общий размер данных: 29815
Тренировочная выборка: 23852 записей (80.0%)
Валидационная выборка: 5963 записей (20.0%)


In [42]:
val_df.describe()

,text,title
count,5963,5963
unique,5963,5963
top,александр казбеги один из самых популярных гру...,хевисбери гоча
freq,1,1


Теперь в валидации нет повторяющихся заголовков.

In [43]:
val_df.to_csv("../data/training_data/val_df.csv", index=False)

val_df.to_csv("../data/val_df.csv", index=False)# Аугментация данных

Аугментировать будем только train набор, чтобы не произошла утечка.

In [44]:
train_df.reset_index(drop=True, inplace=True)

In [45]:
def split_text_into_chunks(text, sentences_per_chunk=3):
    sentences = sent_tokenize(text, language="russian")
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk])
        chunks.append(chunk)
    return chunks

In [46]:
augmented_rows = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    chunks = split_text_into_chunks(row.text, sentences_per_chunk=3)
    for chunk in chunks:
        augmented_rows.append({"text": chunk, "title": row.title})

100%|██████████| 23852/23852 [00:24<00:00, 993.13it/s] 


In [47]:
augmented_dataset = pd.DataFrame(augmented_rows)

In [48]:
augmented_dataset.describe()

,text,title
count,1315615,1315615
unique,1287085,21313
top,. . .,стелла
freq,917,9810


## Почистим строки, в которых нет текста и удалим лидирующую пунктуацию

In [49]:
def is_meaningful(text):
    return bool(re.search(r"[А-Яа-яA-Za-z0-9]", text))

def clean_leading_punct(text):
    return re.sub(r"^[^\wА-Яа-я0-9]+", "", text).strip()

In [50]:
augmented_dataset = augmented_dataset[augmented_dataset.text.apply(is_meaningful)].reset_index(drop=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"[^\w\s,.!?-]", " ", regex=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"\s+", " ", regex=True).str.strip()
augmented_dataset.describe()

,text,title
count,1312611,1312611
unique,1286111,21313
top,если же кого-то заинтересует более подробно то...,стелла
freq,50,9810


Удалим строки, с небольшим количеством данных

In [51]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.strip().str.len() > 10]

In [52]:
augmented_dataset.describe()

,text,title
count,1312016,1312016
unique,1285698,21306
top,если же кого-то заинтересует более подробно то...,стелла
freq,50,9810


Заметим, что у нас очень много текстов с одинаковыми названиями. Это может плохо повлиять на модель, если она будет видеть одни и те же названия. Оставим только по 500 каждого

In [53]:
max_per_title = 500
augmented_dataset = augmented_dataset.groupby("title").head(max_per_title).reset_index(drop=True)

print(augmented_dataset.title.value_counts().head(10))

title
полет                                        500
и был вечер, и было утро                     500
яростная калифорния                          500
по темным заводям. круг замыкается           500
снежный зверь                                500
дожила до понедельника                       500
экспериментповесть                           500
отряд состоит из кота                        500
случай на вокзале                            500
нашествие арабуру третий роман о натабуре    500
Name: count, dtype: int64


In [54]:
augmented_dataset.describe()

,text,title
count,839599,839599
unique,824908,21306
top,"это не значит, что остальным жителям планеты н...",полет
freq,50,500


Появились одинаковые тексты

In [55]:
augmented_dataset.duplicated().sum()

np.int64(1799)

Есть полные дубликаты текст + название. Такое удалим.

In [56]:
augmented_dataset.drop_duplicates(inplace=True)

In [57]:
augmented_dataset.reset_index(drop=True, inplace=True)

In [58]:
augmented_dataset.describe()

,text,title
count,837800,837800
unique,824908,21306
top,если же кого-то заинтересует более подробно то...,если солнечные часы отстают
freq,50,500


Все еще остались одинаковые тексты, но теперь у них разные названия. Удалим и их.

In [59]:
augmented_dataset = augmented_dataset[~augmented_dataset.text.duplicated(keep=False)].reset_index(drop=True)

In [60]:
augmented_dataset.describe()

,text,title
count,821612,821612
unique,821612,21302
top,"бабушка обещала купить пряник, если внук собер...",хан
freq,1,500


In [61]:
augmented_dataset

,text,title
0,"бабушка обещала купить пряник, если внук собер...",конь с розовой гривой
1,"сказка бочки один из первых памфлетов, написан...",сказка бочки
2,", была включена папой римским в . досталось св...",сказка бочки
3,"примечательно, впрочем, что, при полном отсутс...",сказка бочки
4,построение памфлета на первый взгляд может пок...,сказка бочки
...,...,...
821607,у единственного удара бутылкой оказались слишк...,аннотации к моим произведениям
821608,"..стесняться писать?..да много ли поэтов, суме...",ответы в письмах. предисловие
821609,..позволить людям как глоток живой воды испить...,ответы в письмах. предисловие
821610,"в цирке.на сцене большая, даже огромная, грязн...","тот, кто получает пощечины"


In [67]:
augmented_dataset.to_csv("../data/training_data/train_df.csv", index=False)

## Посмотрим на распределение названий по длине после разделения

In [63]:
train_df = augmented_dataset.copy()

In [64]:
dup_titles = (
    augmented_dataset.groupby("text")["title"]
    .nunique()
    .reset_index()
    .query("title > 1")
)

print(f"Текстов с одинаковыми содержаниями, но разными названиями: {len(dup_titles)}")

Текстов с одинаковыми содержаниями, но разными названиями: 0


In [65]:
data["title_len"] = data.title.apply(lambda x: len(str(x).split()))
length_counts = data.groupby('title_len').size()
train_df["title_len"] = train_df.title.apply(lambda x: len(str(x).split()))
length_counts_aug = train_df.groupby('title_len').size()
val_df["title_len"] = val_df.title.apply(lambda x: len(str(x).split()))
length_counts_aug_val = val_df.groupby('title_len').size()

/var/folders/wl/_8h74t5s5hz2ntrny0kjx_pw0000gn/T/ipykernel_39747/4038663461.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=length_counts.index,
    y=length_counts.values,
    name="Без аугментации",
    marker_color="blue"
))

fig.add_trace(go.Bar(
    x=length_counts_aug.index,
    y=length_counts_aug.values,
    name="train после аугментации",
    marker_color="orange"
))

fig.add_trace(go.Bar(
    x=length_counts_aug_val.index,
    y=length_counts_aug_val.values,
    name="test после аугментации",
    marker_color="green"
))

fig.update_layout(
    title="Сравнение распределения длин названий до и после аугментации",
    xaxis_title="Длина названия (слов)",
    yaxis_title="Количество названий",
    barmode="group",
    bargap=0.2,
    bargroupgap=0.1,
    width=1000,
    height=500
)

fig.show()